In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import csv
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import SGDClassifier

In [2]:
readIn = pd.read_csv("data/df_dummy_prepped.csv")
df = pd.DataFrame(readIn)

In [3]:
readIn = pd.read_csv("data/training_data.csv")
df_og = pd.DataFrame(readIn)


In [4]:
cols_to_keep=['CONTRACT_ID','BORROWER_ID', 'CONTRACT_REFINANCED']
df_og=df_og[cols_to_keep]

In [5]:
df_og

,CONTRACT_ID,BORROWER_ID,CONTRACT_REFINANCED
0,TpK8osXs,d8SqtuEV,1.0
1,EtIEHrcH,lrdxML0g,NaN
2,1G10DfKj,gII7nnq4,2.0
3,2NLT774,MMkJ8z/e,NaN
4,VpylRvay,M417onFP,2.0
...,...,...,...
1602748,5B25613A,Z9fieAgO,NaN
1602749,947A5B38,26ZlUKOa,NaN
1602750,947A5B38,Dxng1cGs,NaN
1602751,46115AEC,Ggu/Vok7,NaN


In [6]:
merged_df = pd.merge(df, df_og, on=['CONTRACT_ID', 'BORROWER_ID'], how='left')


In [7]:
del df
del df_og

In [8]:
input=list(merged_df.columns)

In [9]:
merged_df ['CONTRACT_REFINANCED'].value_counts()

CONTRACT_REFINANCED
2.0        1405958
1.0         119039
10001.0      17374
0.0            960
Name: count, dtype: int64

In [10]:
to_remove=['CONTRACT_ID', 'BORROWER_ID', 'CONTRACT_DATE_OF_LOAN_AGREEMENT','CONTRACT_MATURITY_DATE', 'TARGET_EVENT_DAY', 'TARGET_EVENT']

In [11]:
new_input = [x for x in input if x not in to_remove]

In [12]:
new_input.remove('CONTRACT_REFINANCED_0.0')
new_input.remove('CONTRACT_REFINANCED_1.0')
new_input.remove('CONTRACT_REFINANCED_2.0')
new_input.remove('CONTRACT_REFINANCED_10001.0')

In [13]:
class_df=merged_df.dropna(subset=['CONTRACT_REFINANCED'], axis=0)

In [14]:
target= 'CONTRACT_REFINANCED'

In [15]:
new_input.remove('CONTRACT_REFINANCED')

In [16]:
X = class_df[new_input]
y = class_df[target]

In [17]:
np.unique(y)

array([0.0000e+00, 1.0000e+00, 2.0000e+00, 1.0001e+04])

In [18]:
y.isna().sum()

0

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [20]:
del X
del y

In [21]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [22]:
del X_train
del y_train

In [23]:
from sklearn.metrics import accuracy_score

In [24]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

In [25]:
print(f'Model pontossága: {accuracy}')

Model pontossága: 0.9465185895525289


In [26]:
del y_pred
del y_test
del X_test

In [27]:
merged_df['ref']=model.predict(merged_df[new_input])

In [28]:
merged_df['ref'].value_counts()

ref
2.0        1447100
1.0         114019
0.0          24221
10001.0      17413
Name: count, dtype: int64

In [29]:
merged_df['CONTRACT_REFINANCED'].value_counts()

CONTRACT_REFINANCED
2.0        1405958
1.0         119039
10001.0      17374
0.0            960
Name: count, dtype: int64

In [30]:
# merged_df=merged_df.drop('CONTRACT_REFINANCED', axis=1)
merged_df=merged_df.drop('CONTRACT_REFINANCED_0.0', axis=1)
merged_df=merged_df.drop('CONTRACT_REFINANCED_1.0', axis=1)
merged_df=merged_df.drop('CONTRACT_REFINANCED_2.0', axis=1)
merged_df=merged_df.drop('CONTRACT_REFINANCED_10001.0', axis=1)

In [31]:
merged_df['CONTRACT_REFINANCED'] = merged_df.apply(lambda row: row['ref'] if pd.isna(row['CONTRACT_REFINANCED']) else row['CONTRACT_REFINANCED'], axis=1)
merged_df=merged_df.drop('ref', axis=1)

In [32]:
dummy_cols=[]

def dummy_creator(df, column, dummy_col):

  ps_dummies = pd.get_dummies(df[column], prefix = column, drop_first = False)


  dummy_col.append(ps_dummies.columns)

  df = df.merge(ps_dummies, left_index = True, right_index = True, how = 'left')

  return df, dummy_col

In [33]:
# merged_df.rename(columns={'ref': 'CONTRACT_REFINANCED'}, inplace=True)

In [34]:
merged_df['CONTRACT_REFINANCED'].value_counts()

CONTRACT_REFINANCED
2.0        1441668
1.0         119490
0.0          24221
10001.0      17374
Name: count, dtype: int64

In [35]:
merged_df, dummy_cols=dummy_creator(merged_df,'CONTRACT_REFINANCED', dummy_cols)
merged_df=merged_df.drop('CONTRACT_REFINANCED', axis=1)

In [36]:
merged_df.to_csv('data/df_dummy_prepped-2-1.csv', index=False)